In [1]:
# This dataset has more than 30 variables, each with different numerical distributions and categorical variables, as well as some dichotomous variables that will need transformation—this dataset truly has it all. By the end of this section, you'll have built a robust deep learning model that is capable of training and classifying on a complex real-world dataset!

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('Resources/HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
#v Looking at the top of our DataFrame, there are multiple columns with categorical values as well as our numerical values. To make things easier, we should generate a list of categorical variable names. Instead of searching across all 35 columns and keeping track of which variables need categorical preprocessing, we'll let Python do all of the heavy lifting. As an added bonus, we can use our variable list to perform the one-hot encoding once, rather than for each individual variable. To generate our variable list, we'll use Pandas Dataframe.dtypes property. Add and run the following code in your notebook:

# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [4]:
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [5]:
# Now that we have our variable names separated, we can start to preprocess our data starting with the one-hot encoding of the categorical data. Looking at our attrition_cat variable, there are eight categorical variables that need encoding. However, before we loop through our variables and encode them using Scikit-learn's OneHotEncoder module, we need to make sure that none of the categorical variables have more than 10 unique values and require bucketing. To check the three variables, add and run the following code:

# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [6]:
# According to the nunique method, none of the categorical variables have more than 10 unique values, which means we're ready to encode. Previously, we have encoded a single variable at a time, but Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our attrition_cat variable list. Again, add and run the following code to your notebook:

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [7]:
# Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset.

# To replace these columns, we'll use a combination of Pandas' merge and drop methods. Add and run the following code in the notebook:

# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
# Next we must split our training and testing data, then standardize our numerical variables using Scikit-learn's StandardScaler module.

# REWIND
# We need to split our training and testing data before fitting our StandardScaler instance. This prevents testing data from influencing the standardization function.

# To build our training and testing datasets, we need to separate two values:

# input values (which are our independent variables commonly referred to as model features or "X" in TensorFlow documentationLinks to an external site.)
# target output (our dependent variable commonly referred to as target or "y" in TensorFlow documentation)
# For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the "Attrition" columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the "Attrition_Yes" column—we can ignore the "Attrition_No" column because it is redundant. To separate our features and target as well as perform our training/test split, add and run the following code in your notebook:

# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [9]:
# Now that our training and testing data have been allocated, we're ready to build our StandardScalerobject and standardize the numerical features. Add and run the following code to the notebook:

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Now that our data is preprocessed via one-hot encoding and standardization, we should probably perform a gut check to ensure that no data has been lost from our original DataFrame.

In [11]:
# At last, our data is preprocessed and separated and ready for modelling. For our purposes, we will use the same framework we used for our basic neural network:

# For our input layer, we must add the number of input features equal to the number of variables in our feature DataFrame.
# In our hidden layers, our deep learning model structure will be slightly different—we'll add two hidden layers with only a few neurons in each layer. To create the second hidden layer, we'll add another Keras Dense class while defining our model. All of our hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
# In the output layer, we'll use the same parameters from our basic neural network including the sigmoid activation function. The sigmoid activation function will help us predict the probability that an employee is at risk for attrition.
# To create our deep learning model, we must add and run the following code:

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


2024-08-28 04:24:25.529079: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 04:24:25.531631: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [12]:
# Looking at our model summary, we can see that the number of weight parameters (weight coefficients) for each layer equals the number of input values times the number of neurons plus a bias term for each neuron. Our first layer has 55 input values, and multiplied by the eight neurons (plus eight bias terms for each neuron) gives us a total of 448 weight parameters—plenty of opportunities for our model to find trends in the dataset.

# Now it is time to compile our model and define the loss and accuracy metrics. Since we want to use our model as a binary classifier, we'll use the binary_crossentropy loss function, adam optimizer, and accuracy metrics, which are the same parameters we used for our basic neural network. To compile the model, add and run the following code:

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Training and evaluating the deep learning model is no different from a basic neural network. Depending on the complexity of the dataset, we may opt to increase the number of epochs to allow for the deep learning model more opportunities to optimize the weight coefficients. To train our model, we must add and run the following code:

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 1102 samples
Epoch 1/100
1102/1102 [==============================] - 1s 580us/sample - loss: 208.0997 - accuracy: 0.6996
Epoch 2/100
1102/1102 [==============================] - 0s 32us/sample - loss: 60.3220 - accuracy: 0.6289
Epoch 3/100
1102/1102 [==============================] - 0s 31us/sample - loss: 39.8974 - accuracy: 0.6906
Epoch 4/100
1102/1102 [==============================] - 0s 32us/sample - loss: 28.3189 - accuracy: 0.6779
Epoch 5/100
1102/1102 [==============================] - 0s 32us/sample - loss: 19.7262 - accuracy: 0.6942
Epoch 6/100
1102/1102 [==============================] - 0s 31us/sample - loss: 14.3559 - accuracy: 0.7060
Epoch 7/100
1102/1102 [==============================] - 0s 30us/sample - loss: 8.1996 - accuracy: 0.7051
Epoch 8/100
1102/1102 [==============================] - 0s 32us/sample - loss: 9.3060 - accuracy: 0.6915
Epoch 9/100
1102/1102 [==============================] - 0s 31us/sample - loss: 9.0937 - accuracy: 0.7314
Epoch 10/100
110

1102/1102 [==============================] - 0s 29us/sample - loss: 1.1381 - accuracy: 0.7949
Epoch 78/100
1102/1102 [==============================] - 0s 28us/sample - loss: 2.0264 - accuracy: 0.7632
Epoch 79/100
1102/1102 [==============================] - 0s 28us/sample - loss: 1.8824 - accuracy: 0.7940
Epoch 80/100
1102/1102 [==============================] - 0s 28us/sample - loss: 1.7105 - accuracy: 0.7868
Epoch 81/100
1102/1102 [==============================] - 0s 29us/sample - loss: 1.9922 - accuracy: 0.7750
Epoch 82/100
1102/1102 [==============================] - 0s 29us/sample - loss: 1.3925 - accuracy: 0.7813
Epoch 83/100
1102/1102 [==============================] - 0s 29us/sample - loss: 1.5790 - accuracy: 0.7913
Epoch 84/100
1102/1102 [==============================] - 0s 28us/sample - loss: 2.1792 - accuracy: 0.7686
Epoch 85/100
1102/1102 [==============================] - 0s 30us/sample - loss: 2.5241 - accuracy: 0.7713
Epoch 86/100
1102/1102 [==========================

In [14]:
# Lastly, now that our deep learning model is properly trained, we can evaluate the model's performance by testing its predictive capabilities on our testing dataset. Add and run the following code to your notebook:

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.4559 - accuracy: 0.8587
Loss: 0.7638975252275881, Accuracy: 0.8586956262588501


In [15]:
# Looking at our deep learning model's performance metrics, the model was able to correctly identify employees who are at risk of attrition approximately 86% of the time. Considering that our input data included more than 30 different variables with more than 1,400 data points, the deep learning model was able to produce a fairly reliable classifier.

# SKILL DRILL
# Go ahead and test some different neural network designs. You can even test some different machine learning models. How does the performance increase or decrease using different models on the same input data?